Import module and library used

In [1]:
import pandas as pd
import json
import os
import uuid
from icecream import ic
import datetime
import re

Call directory path business_details

In [2]:
directory = 'business_details'

Determine each column in business_details 

In [3]:
columns = ["id", "claim_by", "created_by", "province_id", "province_name", "regency_id", "regency_name", "district_id", "district_name", "village_id", "village_name", "username", "name", "latitude", "longitude", "address", "postal_code", "avatar", "description", "released_at", "google_maps_rating", "created_at", "updated_at", "deleted_at", "is_monday_open", "monday_start_time", "monday_end_time", "monday_notes", "is_tuesday_open", "tuesday_start_time", "tuesday_end_time", "tuesday_notes", "is_wednesday_open", "wednesday_start_time", "wednesday_end_time", "wednesday_notes", "is_thursday_open", "thursday_start_time", "thursday_end_time", "thursday_notes", "is_friday_open", "friday_start_time", "friday_end_time", "friday_notes", "is_saturday_open", "saturday_start_time", "saturday_end_time", "saturday_notes", "is_sunday_open", "sunday_start_time", "sunday_end_time", "sunday_notes", "place_id", "added_from_system", "google_maps_link", "categories"]

Cleaning and extracting each feature from JSON file to CSV file

In [4]:
businesses = pd.DataFrame(columns=columns)
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r')as file:
            json_data = file.read()

        try:
            data = json.loads(json_data)
            result = data["result"]
        except json.JSONDecodeError as e:
            ic("JSON string is not valid:", e)
            ic(f"file: {filename}")
            os.remove(file_path)
            continue

        # if filename == 'ChIJ_XHpnKPl2S0Rp02-zEYeSbI.json':
        #     ic(data)
        
        for item in result:
            id = str(uuid.uuid4())
            name = result["name"]
            address = result["vicinity"]
            latitude = result["geometry"]["location"]["lat"]
            longitude = result["geometry"]["location"]["lng"]
            rating = result.get("rating", 0)
            categories = ", ".join(result.get("types", []))
            opening_hours = result.get("opening_hours", {})
            url = result["url"]

        for component in result["address_components"]:
            if "administrative_area_level_1" in component["types"]:
                province = component["long_name"]
                if "Java" in province:
                    if "East Java" in province:
                        province = "Jawa Timur"
                    elif "Central Java" in province:
                        province = "Jawa Tengah"
                elif "Yogyakarta" in province:
                    province = "DI Yogyakarta"
            elif "administrative_area_level_2" in component["types"]:
                regency = component["long_name"]
                if "Regency" in regency:
                    regency = regency.replace("Regency ", "")
                    regency = "Kabupaten " + district
                elif "City" in regency:
                    regency = regency.replace(" City", "")
                    regency = "Kota " + district
            elif "administrative_area_level_3" in component["types"]:
                district = component["long_name"].replace("Kecamatan ", "")
            elif "administrative_area_level_4" in component["types"]:
                village = component["long_name"]
            elif "postal_code" in component["types"]:
                postal_code = component["long_name"]
        
        created_at = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        place_id = result["place_id"]

        new_business = {'id': id,
                        'claim_by': None,
                        'created_by': None,
                        'province_id': None,
                        'province_name': province,
                        'regency_id': None,
                        'regency_name': regency,
                        'district_id': None,
                        'district_name': district,
                        'village_id': None,
                        'village_name': village,
                        'username': None,
                        'name': name,
                        'latitude': latitude,
                        'longitude': longitude,
                        'address': address,
                        'postal_code': postal_code,
                        'avatar': None,
                        'description': None,
                        'released_at': None,
                        'google_maps_rating': rating,
                        'created_at': created_at,
                        'updated_at': None,
                        'deleted_at': None,
                        'place_id': place_id,
                        'added_from_system': True,
                        'google_maps_link': url,
                        'categories': categories
                        }
        
        days_open = {}
        weekday_text = opening_hours["weekday_text"] if opening_hours else {}
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        for i, day in enumerate(days):
            open_key = f'is_{day.lower()}_open'
            start_key = f'{day.lower()}_start_time'
            end_key = f'{day.lower()}_end_time'
            notes_key = f'{day.lower()}_notes'
            if weekday_text:
                data_day = weekday_text[i]
                is_open = True if re.search(r'–\u2009', data_day) else False
                try:
                    start_time, end_time = re.findall(r'\d{1,2}:\d{2}\u202f\w{2}', data_day)
                    start_time_hour = int(re.findall(r'\d{1,2}', start_time)[0])
                    end_time_hour = int(re.findall(r'\d{1,2}', end_time)[0])

                    if 'PM' in start_time:
                        start_time_hour += 12
                    if 'PM' in end_time:
                        end_time_hour += 12

                    start_time = '{:02d}:{}'.format(start_time_hour, re.findall(r':\d{2}', start_time)[0][1:3])
                    end_time = '{:02d}:{}'.format(end_time_hour, re.findall(r':\d{2}', end_time)[0][1:3])
                except:
                    start_time = None
                    end_time = None
                
                days_open[open_key] = is_open
                days_open[start_key] = start_time
                days_open[end_key] = end_time
                days_open[notes_key] = None
            else:
                days_open[open_key] = None
                days_open[start_key] = None
                days_open[end_key] = None
                days_open[notes_key] = None
        
        new_business.update(days_open)
        businesses = businesses.append(new_business,  ignore_index=True)

In [5]:
businesses.to_csv('businesses.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fa0ff5c2-b244-406d-b1f4-a2f4cd653e27' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>